In [1]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten,Dense
from tensorflow.keras.optimizers import Adam,RMSprop
from sklearn.model_selection import StratifiedKFold
import matplotlib.pyplot as plt
import sys
sys.path.append("..")
from common_functions import image_data_generation


In [2]:
dataset_path = '../images/raw_pollub/'
params = {'dim': (100,100),
          'batch_size': 64,
          'n_classes': 3,
          'n_channels': 3
        }
X,y_categorical,y = image_data_generation(dataset_path,params) 

In [3]:
# StratifiedKFold with k folds
k = 3
skf = StratifiedKFold(n_splits=k)

In [4]:
learning_rate=2e-5
learning_rate2=5e-6

In [5]:
# Liste pour stocker les scores de validation
scores = {i:{'history':None,'history_fine_tuning':None,'score_test':None,'confusion_matrix':None} for i in range(k)}

def eval_cnn(i,train_index,val_test_index):
    #Train dataset
    X_train = X[train_index]
    y_train = y[train_index]

    #Validation and test dataset
    skf2 = StratifiedKFold(n_splits=2)
    test_index,val_index = next(skf2.split(X[val_test_index], y[val_test_index]))
    X_val = X[val_test_index][val_index]
    y_val = y[val_test_index][val_index]
    X_test = X[val_test_index][test_index]
    y_test = y[val_test_index][test_index]

    y_train = keras.utils.to_categorical(y_train, num_classes=params['n_classes'])
    y_val = keras.utils.to_categorical(y_val, num_classes=params['n_classes'])
    y_test = keras.utils.to_categorical(y_test, num_classes=params['n_classes'])

    #VGG16 model for feature extraction
    base_model = VGG16(weights='imagenet', include_top=False, input_shape=(params['dim'][0], params['dim'][1],params['n_channels']))

    #Freeze the layers of the convolutional base
    for layer in base_model.layers:
        layer.trainable = False

    #create the top layers for feature extraction
    model = Sequential()
    model.add(base_model)
    model.add(Flatten())
    model.add(Dense(256, activation='relu'))
    model.add(Dense(params['n_classes'], activation='softmax'))

    model.compile(optimizer=Adam(learning_rate=learning_rate), loss='categorical_crossentropy', metrics=['accuracy'])

    #Train model
    print(f'Fold {i}')
    scores[i]['history'] = model.fit(x=X_train,y=y_train,validation_data=(X_val,y_val),epochs=50)

    #Unfreeze the upper layers of the convolutional base
    for layer in model.layers[0].layers[15:]:
        layer.trainable = True
    
    #Compile the model for fine-tuning
    model.compile(optimizer=Adam(learning_rate=learning_rate2), loss='categorical_crossentropy', metrics=['accuracy'])

    #Train model fine tuning
    print(f'Fold {i} fine tuning')
    scores[i]['history_fine_tuning'] = model.fit(x=X_train,y=y_train,validation_data=(X_val,y_val),epochs=10)

    scores[i]['score_test'] = model.evaluate(X_test, y_test)
    y_pred = model.predict(X_test)
    scores[i]['confusion_matrix'] = tf.math.confusion_matrix(np.argmax(y_test,axis=1),np.argmax(y_pred,axis=1))
    return

In [6]:
#Cross-validation
for i, (train_index, val_test_index) in enumerate(skf.split(X, y)):
    eval_cnn(i,train_index,val_test_index)

Fold 3
Epoch 1/50
118/118 [==============================] - 17s 103ms/step - loss: 0.6432 - accuracy: 0.7390 - val_loss: 0.4458 - val_accuracy: 0.8658
Epoch 2/50
118/118 [==============================] - 9s 76ms/step - loss: 0.3984 - accuracy: 0.8997 - val_loss: 0.3724 - val_accuracy: 0.9026
Epoch 3/50
118/118 [==============================] - 9s 76ms/step - loss: 0.3417 - accuracy: 0.9117 - val_loss: 0.3355 - val_accuracy: 0.8802
Epoch 4/50
118/118 [==============================] - 9s 75ms/step - loss: 0.2997 - accuracy: 0.9202 - val_loss: 0.2979 - val_accuracy: 0.9010
Epoch 5/50
118/118 [==============================] - 9s 76ms/step - loss: 0.2706 - accuracy: 0.9236 - val_loss: 0.2687 - val_accuracy: 0.9105
Epoch 6/50
118/118 [==============================] - 9s 76ms/step - loss: 0.2461 - accuracy: 0.9305 - val_loss: 0.2662 - val_accuracy: 0.8850
Epoch 7/50
118/118 [==============================] - 9s 77ms/step - loss: 0.2268 - accuracy: 0.9324 - val_loss: 0.2309 - val_accurac

In [8]:
#Save results
import os
path = 'Results/'
optimizer='raw_Adam'
output_path = path + f"{optimizer}_k={k}_{learning_rate}_{learning_rate2}"
if not os.path.exists(output_path):
    os.makedirs(output_path)
for i in range(k):
    for metric in ['accuracy','val_accuracy','loss','val_loss']:
        np.save(f"{output_path}/history_{i}_{learning_rate}_{learning_rate2}_{metric}.npy",np.concatenate((scores[i]['history'].history[metric],scores[i]['history_fine_tuning'].history[metric])))
    for j,metric in enumerate(['test_loss','test_accuracy']):
        np.save(f"{output_path}/value_{i}_{learning_rate}_{learning_rate2}_{metric}.npy",scores[i]['score_test'][j])
    np.save(f"{output_path}/value_{i}_{learning_rate}_{learning_rate2}_confusion_matrix.npy",scores[i]['confusion_matrix'])